# 01 — Preprocessing & EDA (Beijing Multi-Site Air Quality)
Mục tiêu: tải dữ liệu, làm sạch, tạo nhãn phân lớp (AQI class theo PM2.5 24h mean), tạo đặc trưng thời gian + lag, và lưu `data/processed/cleaned.parquet`.

**Lưu ý:** nếu `USE_UCIMLREPO=True` thì notebook cần internet để tải dataset từ UCI.

In [ ]:
USE_UCIMLREPO = False
RAW_ZIP_PATH = "data/raw/PRSA2017_Data_20130301-20170228.zip"

OUTPUT_CLEANED_PATH = 'data/processed/cleaned.parquet'
LAG_HOURS=[1, 3, 24]


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

from src.classification_library import (
    load_beijing_air_quality,
    clean_air_quality_df,
    add_pm25_24h_and_label,
    add_time_features,
    add_lag_features,
)

PROJECT_ROOT = Path('..').resolve()
OUT_PATH = (PROJECT_ROOT / OUTPUT_CLEANED_PATH).resolve()
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)


In [ ]:
df_raw = load_beijing_air_quality(use_ucimlrepo=USE_UCIMLREPO, raw_zip_path=RAW_ZIP_PATH)
print('raw shape:', df_raw.shape)
df_raw.head()

In [ ]:
df = clean_air_quality_df(df_raw)
df = add_pm25_24h_and_label(df)
df = add_time_features(df)
df = add_lag_features(df, lag_hours=LAG_HOURS)
print('cleaned shape:', df.shape)
df[['datetime','station','PM2.5','pm25_24h','aqi_class']].head(10)

In [ ]:
# Q1.1.1 Kiểm tra khoảng thời gian dữ liệu phủ (start/end)
df['datetime'].min(), df['datetime'].max()

# Q1.1.2 Kiểm tra tần suất theo giờ và tính liên tục (theo 1 station mẫu)
sample_station = df['station'].iloc[0]
df_station = df[df['station'] == sample_station].set_index('datetime')

# Tần suất theo giờ
pd.infer_freq(df_station.index)

# Kiểm tra số mốc giờ bị thiếu
full_index = pd.date_range(
    start=df_station.index.min(),
    end=df_station.index.max(),
    freq='H'
)

len(full_index.difference(df_station.index))

In [ ]:
# Q1.2.1: Tỷ lệ thiếu theo biến
# EDA nhanh: missingness và phân bố lớp
missing_rate = df.isna().mean().sort_values(ascending=False)
missing_rate.head(20)

# Q1.2.2 Thiếu dữ liệu PM2.5 theo thời gian
df.set_index('datetime')['PM2.5'].isna().resample('M').mean().plot(
    figsize=(10,4),
    title='Monthly missing rate of PM2.5'
)

In [ ]:
# Q1.3 Dùng boxplot hoặc quantile để nhìn nhanh ngoại lai (outliers) và phân phối lệch.
df['PM2.5'].describe(percentiles=[0.01, 0.05, 0.95, 0.99])
df[['PM2.5']].boxplot(figsize=(4,6))

In [ ]:
# Q1.4: Vẽ chuỗi PM2.5 theo thời gian
df_station['PM2.5'].plot(
    figsize=(12,4),
    title=f'PM2.5 over time - station {sample_station}'
)

# Phóng to 1–2 tháng
df_station.loc['2014-01':'2014-02','PM2.5'].plot(
    figsize=(12,4),
    title='PM2.5 (Zoomed 1–2 months)'
)

In [ ]:
# Q1.5: Tự tương quan với các độ trễ 24h & 168h
df_station['PM2.5'].corr(df_station['PM2.5'].shift(24)), \
df_station['PM2.5'].corr(df_station['PM2.5'].shift(168))

In [ ]:
# Q1.6: Kiểm tra tính dừng (ADF / KPSS)
from statsmodels.tsa.stattools import adfuller, kpss

series = df_station['PM2.5'].dropna()

adf_result = adfuller(series)
kpss_result = kpss(series, regression='c')

adf_result, kpss_result


In [ ]:
class_dist = df['aqi_class'].value_counts(dropna=False)
class_dist

In [ ]:
import matplotlib.pyplot as plt

class_dist.drop(index=[x for x in class_dist.index if pd.isna(x)], errors='ignore').plot(kind='bar')
plt.title('AQI class distribution (PM2.5 24h mean)')
plt.ylabel('count')
plt.tight_layout()
plt.show()

In [ ]:
df.to_parquet(OUT_PATH, index=False)
print('Saved:', OUT_PATH)